In [ ]:
import music21
import pathlib

from audio_converter import ABCMusicConverter

In [ ]:
ABCMusicConverter("cooleys.abc", "cooleys").to_mp3(
    instrument="flute",
    tempo=190,
    max_notes=300,
    cut_silence=30,
    start=0.57,
    noise_amplitude=0.001,
    duration=120,
    vbr=8,
    clean_files=True
)

# CLAP

In [1]:
import torch
from laion_clap import CLAP_Module

# Load pretrained CLAP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLAP_Module(enable_fusion=False)  # disable fusion for audio-only use
model.load_ckpt()  # downloads pretrained weights
model = model.to(device)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maxime/.venvs/session/lib64/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm2.we

In [2]:
import torchaudio

waveform, sr = torchaudio.load("cooleys.mp3")

In [3]:
import torchaudio
import torch


def load_audio(filepath, target_sr=16000):
    waveform, sr = torchaudio.load(filepath)
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, sr, target_sr)
    return waveform.mean(dim=0).unsqueeze(0)  # Convert to mono, add batch dim

In [4]:
audio_tensor0 = load_audio("audio/1_cooleys/36371_0.mp3")
audio_tensor1 = load_audio("audio/1_cooleys/36371_1.mp3")
audio_tensor2 = load_audio("audio/8_the_banshee/32846_0.mp3")
audio_tensor3 = load_audio("audio/8_the_banshee/32846_1.mp3")

In [5]:
with torch.no_grad():
    embedding0 = model.get_audio_embedding_from_data(audio_tensor0, use_tensor=True)
    embedding1 = model.get_audio_embedding_from_data(audio_tensor1, use_tensor=True)
    embedding2 = model.get_audio_embedding_from_data(audio_tensor2, use_tensor=True)
    embedding3 = model.get_audio_embedding_from_data(audio_tensor3, use_tensor=True)

In [6]:
embeddings = torch.cat([embedding0, embedding1, embedding2, embedding3], dim=0)
embeddings.shape

torch.Size([4, 512])

In [7]:
import torch.nn.functional as F

F.cosine_similarity(embeddings[None, :, :], embeddings[:, None, :], dim=-1)

tensor([[1.0000, 0.9245, 0.9079, 0.4600],
        [0.9245, 1.0000, 0.9313, 0.4825],
        [0.9079, 0.9313, 1.0000, 0.4726],
        [0.4600, 0.4825, 0.4726, 1.0000]], device='cuda:0')

In [8]:
import torch
import torch.nn.functional as F


def nt_xent_loss(z1, z2, temperature=0.07):
    """
    Contrastive loss using implicit negatives (NT-Xent).
    Args:
        z1: Tensor of shape (N, D) – embeddings from view 1 (e.g., anchors)
        z2: Tensor of shape (N, D) – embeddings from view 2 (e.g., positives)
    Returns:
        Scalar contrastive loss
    """
    batch_size = z1.size(0)

    # Concatenate for full 2N x D
    z = torch.cat([z1, z2], dim=0)  # shape: (2N, D)

    # Cosine similarity (2N x 2N)
    sim = F.cosine_similarity(z[None, :, :], z[:, None, :], dim=-1)

    # Mask self-similarity
    mask = torch.eye(2 * batch_size, device=z.device).bool()
    sim.masked_fill_(mask, -float("inf"))  # ignore similarity to self

    # Targets: for i in 0..N-1, positive pair is i<->i+N and i+N<->i
    targets = torch.cat(
        [torch.arange(batch_size, 2 * batch_size), torch.arange(0, batch_size)]
    ).to(z.device)

    loss = F.cross_entropy(sim / temperature, targets)
    return loss

In [9]:
loss =nt_xent_loss(embeddings[0:2], embeddings[2:])
loss

tensor(2.4266, device='cuda:0')

In [23]:
import pathlib
import torch
import torchaudio
import numpy as np

class TheSessionDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, sampling_rate=16000, prng=None, buffer_size=4096):
        self.root_dir = pathlib.Path(root_dir)
        self.sampling_rate = sampling_rate
        self.buffer_size = buffer_size

        if isinstance(prng, np.random.Generator):
            self.prng = prng
        else:
            self.prng = np.random.default_rng(prng)

        # Regroup recordings of the same recordss together
        records = {}

        for file in self.root_dir.glob("**/*.mp3"):
            record_id = file.stem.split("_")[0]

            if record_id in records:
                records[record_id].append(file)
            else:
                records[record_id] = [file]

        # Keep only recordings that have 2 or more versions
        self.records = list(r for r in records.values() if len(r) > 1)

    def __len__(self):
        return len(self.records)
    
    def load_audio(self, filepath):
        waveform, sr = torchaudio.load(filepath)
        if sr != self.sampling_rate:
            waveform = torchaudio.functional.resample(waveform, sr, self.sampling_rate, buffer_size=self.buffer_size)
        
        #return waveform.mean(dim=0).unsqueeze(0)
        return waveform.mean(dim=0)

    def __getitem__(self, idx):
        # Select 2 versions of the record randomly (without replacement)
        selected = self.prng.choice(self.records[idx], size=2, replace=False)

        return self.load_audio(selected[0]), self.load_audio(selected[1])
        

In [35]:
d = TheSessionDataset("audio", buffer_size=2**15)
d.records

[[PosixPath('audio/3396_james_browns/4301_0.mp3'),
  PosixPath('audio/3396_james_browns/4301_1.mp3'),
  PosixPath('audio/3396_james_browns/4301_2.mp3'),
  PosixPath('audio/3396_james_browns/4301_3.mp3'),
  PosixPath('audio/3396_james_browns/4301_4.mp3')],
 [PosixPath('audio/3396_james_browns/4302_0.mp3'),
  PosixPath('audio/3396_james_browns/4302_1.mp3'),
  PosixPath('audio/3396_james_browns/4302_2.mp3'),
  PosixPath('audio/3396_james_browns/4302_3.mp3'),
  PosixPath('audio/3396_james_browns/4302_4.mp3')],
 [PosixPath('audio/3396_james_browns/4303_0.mp3'),
  PosixPath('audio/3396_james_browns/4303_1.mp3'),
  PosixPath('audio/3396_james_browns/4303_2.mp3'),
  PosixPath('audio/3396_james_browns/4303_3.mp3'),
  PosixPath('audio/3396_james_browns/4303_4.mp3')],
 [PosixPath('audio/1374_cucanandy/46555_0.mp3'),
  PosixPath('audio/1374_cucanandy/46555_1.mp3'),
  PosixPath('audio/1374_cucanandy/46555_2.mp3'),
  PosixPath('audio/1374_cucanandy/46555_3.mp3'),
  PosixPath('audio/1374_cucanandy/46

In [36]:
d[0]

(tensor([ 0.0030,  0.0040,  0.0027,  ..., -0.0140, -0.0150, -0.0110]),
 tensor([ 0.0007,  0.0009,  0.0010,  ..., -0.0002,  0.0003, -0.0001]))

In [37]:
%%timeit

d[0]

46 ms ± 204 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
data_loader = torch.utils.data.DataLoader(d, batch_size=16, shuffle=True)

In [31]:
%%timeit

z1, z2 = next(iter(data_loader))

866 ms ± 26.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
z1, z2 = next(iter(data_loader))

In [22]:
z1.shape

torch.Size([16, 1920000])

In [38]:
z1_embed = model.get_audio_embedding_from_data(z1, use_tensor=True)

In [39]:
z2_embed = model.get_audio_embedding_from_data(z2, use_tensor=True)

In [40]:
loss = nt_xent_loss(z1_embed, z2_embed)

In [41]:
loss.backward()